In [1]:
import cv2
import numpy as np
import sys
import os
import math
from openvino.inference_engine import IENetwork, IEPlugin

print(cv2.__version__)

3.4.3


### Inference config

In [2]:
#IMG_NAME = "butterfly_GT.bmp"
IMG_NAME = "lenna.bmp"

IMG_OPENCV_NAME = "opencv.png"
IMG_DNN_NAME = "srcnn.png"

INTERPOLATION = cv2.INTER_CUBIC
SCALE = 2
SIZE_CONV = 6

PSNR = True

### OpenVino config

In [3]:
model_xml = "yayoi_srcnn_935_2x_512.xml"
model_bin = "yayoi_srcnn_935_2x_512.bin"

plugin = IEPlugin(device="CPU", plugin_dirs=".")
#plugin.add_cpu_extension('MKLDNNPlugin')

print("Loading network files:\n\t{}\n\t{}".format(model_xml, model_bin))
net = IENetwork.from_ir(model=model_xml, weights=model_bin)

if "CPU" in plugin.device:
    supported_layers = plugin.get_supported_layers(net)
    not_supported_layers = [l for l in net.layers.keys() if l not in supported_layers]
    if len(not_supported_layers) != 0:
        print("Following layers are not supported by the plugin for specified device {}:\n {}". format(plugin.device, ', '.join(not_supported_layers)))
        print("Please try to specify cpu extensions library path in sample's command line parameters using -l " "or --cpu_extension command line argument")
        sys.exit(1)

assert len(net.inputs.keys()) == 1, "Sample supports only single input topologies"
assert len(net.outputs) == 1, "Sample supports only single output topologies"

print("Preparing input blobs")
input_blob = next(iter(net.inputs))
out_blob = next(iter(net.outputs))
net.batch_size = 1

n, c, h, w = net.inputs[input_blob]
images = np.ndarray(shape=(n, c, h, w))


Loading network files:
	yayoi_srcnn_935_2x_512.xml
	yayoi_srcnn_935_2x_512.bin
Preparing input blobs


### Generate OpenCV resized image for reference

In [4]:
img = cv2.imread(IMG_NAME, cv2.IMREAD_COLOR)
shape = img.shape

if PSNR:
    img = cv2.resize(img, (int(shape[1] / SCALE), int(shape[0] / SCALE)), INTERPOLATION)
    img = cv2.resize(img, (shape[1], shape[0]), INTERPOLATION)
else:
    img = cv2.resize(img, (shape[1] * SCALE, shape[0] * SCALE), INTERPOLATION)

cv2.imwrite(IMG_OPENCV_NAME, img, [int(cv2.IMWRITE_PNG_COMPRESSION), 0])

True

### Generate super resolution image by SRCNN

In [5]:
img = cv2.imread(IMG_NAME, cv2.IMREAD_COLOR)
img = cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)
shape = img.shape

if PSNR:
    imgY = cv2.resize(img[:, :, 0], (int(shape[1] / SCALE), int(shape[0] / SCALE)), INTERPOLATION)
    imgY = cv2.resize(imgY, (shape[1], shape[0]), cv2.INTER_CUBIC)
    img[:, :, 0] = imgY
else:
    imgY = cv2.resize(img[:, :, 0], (int(shape[1] * SCALE), int(shape[0] * SCALE)), INTERPOLATION)
    img[:, :, 0] = imgY

exec_net = plugin.load(network=net)

tensorY = np.zeros((1, 1, shape[0], shape[1]), dtype=float)
tensorY[0, 0, :, :] = imgY.astype(float)
images[0] = tensorY

# ================================
tick1 = cv2.getTickCount()
res = exec_net.infer(inputs={input_blob: images})
tick2 = cv2.getTickCount()
# ================================

res = res[out_blob]

for batch, data in enumerate(res):
    data = data * 255
    data[data < 0] = 0
    data[data > 255] = 255
    img[SIZE_CONV: -SIZE_CONV, SIZE_CONV: -SIZE_CONV, 0] = data[:, :]
    img = cv2.cvtColor(img, cv2.COLOR_YCrCb2BGR)
    cv2.imwrite(IMG_DNN_NAME, img, [int(cv2.IMWRITE_PNG_COMPRESSION), 0])


### PSNR calculation

In [6]:
im1 = cv2.imread(IMG_NAME, cv2.IMREAD_COLOR)
im1 = cv2.cvtColor(im1, cv2.COLOR_BGR2RGB)
im2 = cv2.imread(IMG_OPENCV_NAME, cv2.IMREAD_COLOR)
im2 = cv2.cvtColor(im2, cv2.COLOR_BGR2RGB)
im3 = cv2.imread(IMG_DNN_NAME, cv2.IMREAD_COLOR)
im3 = cv2.cvtColor(im3, cv2.COLOR_BGR2RGB)

if PSNR:
    print("opencv:")
    print(cv2.PSNR(im1, im2))
    print("srcnn:")
    print(cv2.PSNR(im1, im3))

opencv:
31.47349297867539
srcnn:
34.754624406334706


### Display image

In [7]:
pltShow = False

if pltShow:    
    from matplotlib import pyplot as plt
    %matplotlib inline
    import seaborn as sns
    
    plt.figure(num='comparison',figsize=(16,16))

    plt.subplot(1,3,1)
    plt.title('origin image')
    plt.imshow(im1)

    plt.subplot(1,3,2)
    plt.title('OpenCV')
    plt.imshow(im2)

    plt.subplot(1,3,3)
    plt.title("srcnn")
    plt.imshow(im3)

### Process time calculation

In [8]:
tick = math.floor( ((tick2 - tick1) * 1000) / cv2.getTickFrequency())

if tick >= 60000:
    mins = math.floor(tick / 60000)
    secs = math.floor((tick - mins * 60000) / 1000)
    msec = tick - mins * 60000 - secs * 1000
    print("Inference processed time: " + str(mins) + " mins " + str(secs) + " secs " + str(msec) + " ms")
elif tick >= 1000:
    secs = math.floor(tick / 1000)
    msec = tick - secs * 1000
    print("Inference processed time: " + str(secs) + " secs " + str(msec) + " ms")
else:
    print("Inference processed time: " + str(tick) + " ms")

Inference processed time: 494 ms
